# bootstrap_words_02_add_frequency_embedding

- read in CSV -> spark df
- scrape word frequencies from API -> joblib dict
- scrape word embeddings from BERT -> joblib dict
- load frequency and embeddings, add to spark df
- NO date added column, just use version
- save as Delta table bronze.words


In [ ]:
%run "./00_setup.ipynb"

In [ ]:
import os
import joblib
from pyspark.sql import functions as F
from pyspark.sql.types import *
from src.constants import WORDLIST_PATH, WORDLIST_TEMP_CSV_FILENAME, WORD_DATA_FILENAME
from src.ngramsutils import get_word_frequencies_threaded
from src.embeddingutils import get_word_embeddings
###
import numpy as np
import os
from src.wordutils import get_letter_set, filter_wordlist
from src.fileutils import word_file_to_set, get_local_path
from src.constants import (WORDS_PKL_FILENAME,
                           WORDS_PARQUET_FILENAME,
                           NGRAMS_API_BASE,
                           NGRAMS_BATCH_SIZE)
from src.ngramsutils import get_word_frequencies

In [ ]:
def collect_word_data_with_checkpoint(words_list: list, 
                                     output_path: str,
                                     batch_size: int = 100,
                                     resume_job: bool = False):
    """
    Collect frequency and embedding data for a list of words with checkpointing.
    Saves final results as a joblib file that can be reused for multiple purposes.
    
    Args:
        words_list: List of words to process
        output_path: Path to save final joblib file with all data
        batch_size: Batch size for API calls (default 100 for API limits)
        resume_job: Whether to resume from existing checkpoint
    
    Returns:
        dict: Contains 'freq_dict' and 'embeddings_dict'
    """
    checkpoint_path = output_path.replace('.joblib', '_checkpoint.joblib')
    
    # Check for existing final output
    if os.path.exists(output_path) and not resume_job:
        print(f"Final output already exists at {output_path}")
        with open(output_path, 'rb') as f:
            return joblib.load(f)
    
    # Initialize or load checkpoint
    freq_dict = {}
    embeddings_dict = {}
    start_batch = 0
    
    if resume_job and os.path.exists(checkpoint_path):
        with open(checkpoint_path, 'rb') as f:
            checkpoint_data = joblib.load(f)
            freq_dict = checkpoint_data.get('freq_dict', {})
            embeddings_dict = checkpoint_data.get('embeddings_dict', {})
            start_batch = checkpoint_data.get('last_batch', 0) + 1
        print(f"Resuming from batch {start_batch} with {len(freq_dict)} words already processed")
    
    total_batches = (len(words_list) + batch_size - 1) // batch_size
    
    try:
        # Process remaining batches
        for batch_idx in range(start_batch, total_batches):
            start_idx = batch_idx * batch_size
            end_idx = min(start_idx + batch_size, len(words_list))
            batch_words = words_list[start_idx:end_idx]
            
            print(f"Processing batch {batch_idx + 1}/{total_batches}: words {start_idx}-{end_idx-1}")
            
            # Get frequencies for this batch
            batch_freq_dict = get_word_frequencies_threaded(batch_words, max_workers=10)
            freq_dict.update(batch_freq_dict)
            
            # Get embeddings for this batch
            batch_embeddings_dict = get_word_embeddings(batch_words)
            embeddings_dict.update(batch_embeddings_dict)
            
            # Save checkpoint after each batch
            # Save embeddings as pure python lists
            checkpoint_data = {
                'freq_dict': freq_dict,
                'embeddings_dict': {k: v.tolist() for k, v in embeddings_dict.items()},
                'last_batch': batch_idx,
                'total_batches': total_batches
            }
            with open(checkpoint_path, 'wb') as f:
                joblib.dump(checkpoint_data, f)
            
            print(f"✅ Completed batch {batch_idx + 1}/{total_batches}")
    
    except Exception as err:
        print(f"Exception occurred! Resume job from batch {batch_idx}")
        print(f"Current progress: {len(freq_dict)} words processed")
        raise err
    
    # Save final output
    # Save embeddings not as numpy
    final_data = {
        'freq_dict': freq_dict,
        'embeddings_dict': {k: v.tolist() for k, v in embeddings_dict.items()},
        'processed_words': list(freq_dict.keys()),
        'total_words': len(freq_dict),
        'metadata': {
            'batch_size': batch_size,
            'total_batches': total_batches
        }
    }
    
    with open(output_path, 'wb') as f:
        joblib.dump(final_data, f)
    
    print(f"✅ Processing complete! Saved {len(freq_dict)} words to {output_path}")
    
    # Clean up checkpoint
    if os.path.exists(checkpoint_path):
        os.remove(checkpoint_path)
        print("✅ Cleaned up checkpoint file")
    
    return final_data

In [ ]:
def apply_word_data_to_spark_df(df, word_data_path: str):
    """
    Apply frequency and embedding data to a Spark DataFrame using broadcast variables.
    
    Args:
        df: Spark DataFrame with a 'word' column
        word_data_path: Path to joblib file created by collect_word_data_with_checkpoint
    
    Returns:
        Spark DataFrame with 'frequency' and 'embedding' columns added
    """
    # Load the word data
    with open(word_data_path, 'rb') as f:
        word_data = joblib.load(f)
    
    freq_dict = word_data['freq_dict']
    embeddings_dict = word_data['embeddings_dict']
    
    print(f"Loaded data for {len(freq_dict)} words from {word_data_path}")
    
    # Create broadcast variables for efficient lookups
    freq_broadcast = spark.sparkContext.broadcast(freq_dict)
    embeddings_broadcast = spark.sparkContext.broadcast(embeddings_dict)
    
    frequency_udf = F.udf(lambda word: freq_broadcast.value.get(word, 0.0), FloatType())
    embedding_udf = F.udf(lambda word: embeddings_broadcast.value.get(word, [0.0] * 768), ArrayType(FloatType()))
    
    # Add the new columns
    result_df = df.withColumn("frequency", frequency_udf(F.col("word"))) \
                 .withColumn("embedding", embedding_udf(F.col("word")))
    
    return result_df

In [ ]:
# Read CSV directly into Spark and collect word data
initial_schema = StructType([
    StructField("word", StringType(), True),
    StructField("letter_set", StringType(), True),
    StructField("version", IntegerType(), True)
])

temp_path = get_local_path(f"{WORDLIST_PATH}/{WORDLIST_TEMP_CSV_FILENAME}")
spark_df = spark.read.csv(temp_path, header=True, schema=initial_schema)

print(f"Read in {spark_df.count()} words from csv.")

In [ ]:
words_list = [row.word for row in spark_df.select("word").collect()]
word_data_path = get_local_path(f"{WORDLIST_PATH}/{WORD_DATA_FILENAME}")

# First run - set resume_job=False
# word_data = collect_word_data_with_checkpoint(
#     words_list, 
#     word_data_path,
#     batch_size=100,
#     resume_job=False
# )

# To resume if it crashes - set resume_job=True
# word_data = collect_word_data_with_checkpoint(
#     words_list, 
#     word_data_path,
#     batch_size=100,
#     resume_job=True
#

In [ ]:
# TODO: Testing purposes only -- remove later
test_words_list = words_list[617:115111:297]
print(f"testing on {len(test_words_list)} words...")
word_data = collect_word_data_with_checkpoint(
    test_words_list, 
    word_data_path,
    batch_size=100,
    resume_job=False
)
### END TODO

In [ ]:
# TODO: Remove testing code
with open(word_data_path, "rb") as f:
    my_word_data = joblib.load(f)

print(len(my_word_data['freq_dict']))
print(len(my_word_data['embeddings_dict']))
print(set([len(emb) for _, emb in my_word_data['embeddings_dict'].items()]))
print(my_word_data['total_words'])
print(len(my_word_data['processed_words']))


In [ ]:
# Apply word data to DataFrame
# enriched_df = apply_word_data_to_spark_df(spark_df, word_data_path)

In [ ]:
# TODO: REMOVE TEST CODE
test_df = spark_df.filter(F.col("word").isin(test_words_list))
test_df.count()

In [ ]:
# TODO: REMOVE TEST CODE
test_enriched_df = apply_word_data_to_spark_df(test_df, word_data_path)

In [ ]:
# TODO: REMOVE TEST CODE
print(test_enriched_df.count())
test_enriched_df.columns

In [ ]:
# TODO: REMOVE TEST CODE
print(test_enriched_df.filter(F.col("embedding").isNotNull()).count())
print(test_enriched_df.filter(F.col("frequency").isNotNull()).count())

In [ ]:
test_enriched_df.printSchema()

In [ ]:
test_enriched_df.select("embedding").show()